# Dataset 2018-2019 Bereinigung und Normalisierung
## Spezialisiertes Modul für Kaggle/Immobilienscout24 Dataset

### Ziel
Bereinigung und Normalisierung des historischen Datasets (2018-2019) in ein standardisiertes Format für die gemeinsame Analyse.

### Input
- `data/raw/Dataset_2018_2019.csv`

### Output
- `data/processed/dataset_2018_2019_normalized.csv`

### Standardisierte Ausgabespalten
- `price`: Normalisierter Preis (Kaltmiete in €)
- `size`: Normalisierte Größe (m²)
- `district`: Berliner Bezirk (standardisiert)
- `rooms`: Anzahl Zimmer
- `year`: Jahr des Datasets (2019)
- `dataset_id`: Eindeutige Dataset-Kennzeichnung (historical)
- `source`: Datenquelle

---
**Teil der modularen Preprocessing-Pipeline**  
**Datum:** 4. Juli 2025  
**Version:** 1.0

## 1. Import Required Libraries

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

# Display configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 20)

print("Bibliotheken erfolgreich importiert!")
print(f"Pandas Version: {pd.__version__}")
print(f"Dataset: 2018-2019 (Kaggle/Immobilienscout24)")

Bibliotheken erfolgreich importiert!
Pandas Version: 2.3.0
Dataset: 2018-2019 (Kaggle/Immobilienscout24)


## 2. Daten laden und erste Analyse

In [2]:
# Lade Dataset 2018-2019
print("=" * 60)
print("DATASET 2018-2019 LADEN UND ANALYSIEREN")
print("=" * 60)

# Lade Rohdaten
df_raw = pd.read_csv('data/raw/Dataset_2018_2019.csv')
print(f"Dataset geladen: {df_raw.shape[0]:,} Zeilen, {df_raw.shape[1]} Spalten")

# Grundlegende Informationen
print(f"\nSpalten: {list(df_raw.columns)}")
print(f"\nDatentypen:")
print(df_raw.dtypes)

# Fehlende Werte
print(f"\nFehlende Werte:")
missing_values = df_raw.isnull().sum()
missing_pct = (missing_values / len(df_raw) * 100).round(2)
for col in missing_values[missing_values > 0].index:
    print(f"  {col}: {missing_values[col]} ({missing_pct[col]}%)")

# Erste 5 Zeilen
print(f"\nErste 5 Zeilen:")
print(df_raw.head())

DATASET 2018-2019 LADEN UND ANALYSIEREN
Dataset geladen: 10,406 Zeilen, 9 Spalten

Spalten: ['regio3', 'street', 'livingSpace', 'baseRent', 'totalRent', 'noRooms', 'floor', 'typeOfFlat', 'yearConstructed']

Datentypen:
regio3              object
street              object
livingSpace        float64
baseRent           float64
totalRent          float64
noRooms            float64
floor              float64
typeOfFlat          object
yearConstructed    float64
dtype: object

Fehlende Werte:
  totalRent: 662 (6.36%)
  floor: 1100 (10.57%)
  typeOfFlat: 804 (7.73%)
  yearConstructed: 1425 (13.69%)

Erste 5 Zeilen:
            regio3                      street  livingSpace  baseRent  totalRent  noRooms  floor    typeOfFlat  yearConstructed
0  Staaken_Spandau           Metropolitan Park        77.00    820.00    1140.00      3.0    0.0  ground_floor              NaN
1        Weißensee      B&ouml;rnestra&szlig;e        62.63    808.00     955.00      2.0    0.0  ground_floor           1918.0

## 3. Spezifische Bereinigung Dataset 2018-2019

In [3]:
# Spezifische Bereinigung für Dataset 2018-2019
print("=" * 60)
print("SPEZIFISCHE BEREINIGUNG DATASET 2018-2019")
print("=" * 60)

# Erstelle Arbeitskopie
df = df_raw.copy()
print(f"Arbeitskopie erstellt: {len(df)} Zeilen")

# 1. Preis-Bereinigung (baseRent)
print(f"\n=== PREIS-BEREINIGUNG ===")
print(f"baseRent - Statistik vor Bereinigung:")
print(f"  Typ: {df['baseRent'].dtype}")
print(f"  Nicht-null Werte: {df['baseRent'].notna().sum()}")
print(f"  Min: {df['baseRent'].min()}, Max: {df['baseRent'].max()}")

# Preis ist bereits numerisch, nur Plausibilitätsprüfung
# Entferne unrealistische Preise (< 100€ oder > 10.000€)
original_count = len(df)
df = df[(df['baseRent'] >= 100) & (df['baseRent'] <= 10000)]
removed_price = original_count - len(df)
print(f"Entfernte unrealistische Preise: {removed_price}")

# 2. Größen-Bereinigung (livingSpace)
print(f"\n=== GRÖSSEN-BEREINIGUNG ===")
print(f"livingSpace - Statistik vor Bereinigung:")
print(f"  Typ: {df['livingSpace'].dtype}")
print(f"  Nicht-null Werte: {df['livingSpace'].notna().sum()}")
print(f"  Min: {df['livingSpace'].min()}, Max: {df['livingSpace'].max()}")

# Größe ist bereits numerisch, nur Plausibilitätsprüfung
# Entferne unrealistische Größen (< 10m² oder > 500m²)
original_count = len(df)
df = df[(df['livingSpace'] >= 10) & (df['livingSpace'] <= 500)]
removed_size = original_count - len(df)
print(f"Entfernte unrealistische Größen: {removed_size}")

# 3. Bezirks-Normalisierung (regio3)
print(f"\n=== BEZIRKS-NORMALISIERUNG ===")
print(f"regio3 - Einzigartige Werte: {df['regio3'].nunique()}")
print(f"Bezirke: {sorted(df['regio3'].unique())}")

# Bezirk-Normalisierung (entferne _Suffix)
def normalize_district_2018_2019(district):
    """Normalisiert Bezirksnamen für Dataset 2018-2019"""
    if pd.isna(district):
        return None
    
    # Entferne Suffix nach Unterstrich
    if '_' in str(district):
        return str(district).split('_')[0]
    
    return str(district)

df['district_normalized'] = df['regio3'].apply(normalize_district_2018_2019)

print(f"Normalisierte Bezirke: {sorted(df['district_normalized'].unique())}")
print(f"Anzahl normalisierte Bezirke: {df['district_normalized'].nunique()}")

# 4. Zimmer-Bereinigung (noRooms)
print(f"\n=== ZIMMER-BEREINIGUNG ===")
print(f"noRooms - Statistik:")
print(f"  Typ: {df['noRooms'].dtype}")
print(f"  Nicht-null Werte: {df['noRooms'].notna().sum()}")
print(f"  Einzigartige Werte: {sorted(df['noRooms'].dropna().unique())}")

# Zimmeranzahl ist bereits numerisch
# Plausibilitätsprüfung (0.5 bis 10 Zimmer)
original_count = len(df)
df = df[(df['noRooms'] >= 0.5) & (df['noRooms'] <= 10)]
removed_rooms = original_count - len(df)
print(f"Entfernte unrealistische Zimmeranzahlen: {removed_rooms}")

print(f"\n✅ Spezifische Bereinigung abgeschlossen")
print(f"Verbleibende Datensätze: {len(df)} (Verlust: {len(df_raw) - len(df)})")

SPEZIFISCHE BEREINIGUNG DATASET 2018-2019
Arbeitskopie erstellt: 10406 Zeilen

=== PREIS-BEREINIGUNG ===
baseRent - Statistik vor Bereinigung:
  Typ: float64
  Nicht-null Werte: 10406
  Min: 0.0, Max: 20000.0
Entfernte unrealistische Preise: 11

=== GRÖSSEN-BEREINIGUNG ===
livingSpace - Statistik vor Bereinigung:
  Typ: float64
  Nicht-null Werte: 10395
  Min: 3.0, Max: 542.53
Entfernte unrealistische Größen: 7

=== BEZIRKS-NORMALISIERUNG ===
regio3 - Einzigartige Werte: 79
Bezirke: ['Adlershof_Treptow', 'Alt_Hohenschönhausen_Hohenschönhausen', 'Altglienicke_Treptow', 'Baumschulenweg_Treptow', 'Biesdorf_Marzahn', 'Blankenburg_Weißensee', 'Bohnsdorf_Treptow', 'Britz_Neukölln', 'Buch_Pankow', 'Buckow_Neukölln', 'Charlottenburg', 'Dahlem_Zehlendorf', 'Falkenberg_Hohenschönhausen', 'Französisch_Buchholz_Pankow', 'Friedenau_Schöneberg', 'Friedrichsfelde_Lichtenberg', 'Friedrichshagen_Köpenick', 'Friedrichshain', 'Frohnau_Reinickendorf', 'Gatow_Spandau', 'Grunewald_Wilmersdorf', 'Grünau_Köpe

## 4. Normalisierung in Standardformat

In [4]:
# Normalisierung in Standardformat
print("=" * 60)
print("NORMALISIERUNG IN STANDARDFORMAT")
print("=" * 60)

# Erstelle normalisiertes Dataset mit Standardspalten
df_normalized = pd.DataFrame()

# Standardspalten zuweisen
df_normalized['price'] = df['baseRent'].astype('float64')
df_normalized['size'] = df['livingSpace'].astype('float64')
df_normalized['district'] = df['district_normalized'].astype('string')
df_normalized['rooms'] = df['noRooms'].astype('float64')
df_normalized['year'] = 2019
df_normalized['dataset_id'] = 'historical'
df_normalized['source'] = 'Kaggle/Immobilienscout24'

# Zusätzliche Spalten aus Original-Dataset beibehalten
df_normalized['street'] = df['street']
df_normalized['floor'] = df['floor']
df_normalized['typeOfFlat'] = df['typeOfFlat']
df_normalized['yearConstructed'] = df['yearConstructed']
df_normalized['totalRent'] = df['totalRent']

print(f"Normalisiertes Dataset erstellt: {len(df_normalized)} Zeilen")
print(f"Standardspalten: {['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source']}")
print(f"Zusätzliche Spalten: {list(df_normalized.columns[7:])}")

# Datenqualität prüfen
print(f"\n=== DATENQUALITÄT NORMALISIERTES DATASET ===")
print(f"Zeilen mit Preis: {df_normalized['price'].notna().sum()}")
print(f"Zeilen mit Größe: {df_normalized['size'].notna().sum()}")
print(f"Zeilen mit Bezirk: {df_normalized['district'].notna().sum()}")
print(f"Zeilen mit Zimmeranzahl: {df_normalized['rooms'].notna().sum()}")

# Statistiken
print(f"\n=== STATISTIKEN ===")
print(f"Preis - Min: {df_normalized['price'].min():.2f}€, Max: {df_normalized['price'].max():.2f}€, Median: {df_normalized['price'].median():.2f}€")
print(f"Größe - Min: {df_normalized['size'].min():.1f}m², Max: {df_normalized['size'].max():.1f}m², Median: {df_normalized['size'].median():.1f}m²")
print(f"Zimmer - Min: {df_normalized['rooms'].min():.1f}, Max: {df_normalized['rooms'].max():.1f}, Median: {df_normalized['rooms'].median():.1f}")

# Bezirksverteilung
print(f"\n=== BEZIRKSVERTEILUNG ===")
district_counts = df_normalized['district'].value_counts()
print(f"Anzahl Bezirke: {len(district_counts)}")
for district, count in district_counts.head(10).items():
    print(f"  {district}: {count} Einträge")

print(f"\n✅ Normalisierung abgeschlossen!")

NORMALISIERUNG IN STANDARDFORMAT
Normalisiertes Dataset erstellt: 10387 Zeilen
Standardspalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source']
Zusätzliche Spalten: ['street', 'floor', 'typeOfFlat', 'yearConstructed', 'totalRent']

=== DATENQUALITÄT NORMALISIERTES DATASET ===
Zeilen mit Preis: 10387
Zeilen mit Größe: 10387
Zeilen mit Bezirk: 10387
Zeilen mit Zimmeranzahl: 10387

=== STATISTIKEN ===
Preis - Min: 178.16€, Max: 9500.00€, Median: 945.00€
Größe - Min: 10.0m², Max: 482.0m², Median: 72.0m²
Zimmer - Min: 1.0, Max: 10.0, Median: 2.0

=== BEZIRKSVERTEILUNG ===
Anzahl Bezirke: 79
  Mitte: 799 Einträge
  Tiergarten: 768 Einträge
  Charlottenburg: 701 Einträge
  Friedrichshain: 553 Einträge
  Prenzlauer: 473 Einträge
  Spandau: 415 Einträge
  Wedding: 397 Einträge
  Wilmersdorf: 370 Einträge
  Neukölln: 361 Einträge
  Köpenick: 351 Einträge

✅ Normalisierung abgeschlossen!


## 5. Export des normalisierten Datasets

In [5]:
# Export des normalisierten Datasets
print("=" * 60)
print("EXPORT NORMALISIERTES DATASET")
print("=" * 60)

# Ausgabedatei
output_file = 'data/processed/dataset_2018_2019_normalized.csv'

# Export
df_normalized.to_csv(output_file, index=False)
print(f"✅ Normalisiertes Dataset exportiert: {output_file}")
print(f"Dateigröße: {len(df_normalized)} Zeilen x {len(df_normalized.columns)} Spalten")

# Validierung des Exports
test_load = pd.read_csv(output_file)
print(f"✅ Export-Validierung erfolgreich: {len(test_load)} Zeilen geladen")

# Zusammenfassung
print(f"\n=== ZUSAMMENFASSUNG DATASET 2018-2019 ===")
print(f"Input: data/raw/Dataset_2018_2019.csv ({len(df_raw)} Zeilen)")
print(f"Output: {output_file} ({len(df_normalized)} Zeilen)")
print(f"Datenverlust: {len(df_raw) - len(df_normalized)} Zeilen ({((len(df_raw) - len(df_normalized))/len(df_raw)*100):.1f}%)")
print(f"Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source")
print(f"Zusätzliche Spalten: {len(df_normalized.columns) - 7}")

print(f"\n🎯 DATASET 2018-2019 BEREINIGUNG ABGESCHLOSSEN!")
print(f"Bereit für Kombination mit anderen normalisierten Datasets.")

EXPORT NORMALISIERTES DATASET
✅ Normalisiertes Dataset exportiert: data/processed/dataset_2018_2019_normalized.csv
Dateigröße: 10387 Zeilen x 12 Spalten
✅ Export-Validierung erfolgreich: 10387 Zeilen geladen

=== ZUSAMMENFASSUNG DATASET 2018-2019 ===
Input: data/raw/Dataset_2018_2019.csv (10406 Zeilen)
Output: data/processed/dataset_2018_2019_normalized.csv (10387 Zeilen)
Datenverlust: 19 Zeilen (0.2%)
Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source
Zusätzliche Spalten: 5

🎯 DATASET 2018-2019 BEREINIGUNG ABGESCHLOSSEN!
Bereit für Kombination mit anderen normalisierten Datasets.
